In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import json
import os
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
import pandas as pd

import re
import requests
from PIL import Image

In [17]:
PATH = "C:/Program Files/Chrome Webdriver"
os.environ['PATH'] += PATH
driver = webdriver.Chrome()
driver.get("https://www.flipkart.com/")

In [20]:
df = pd.DataFrame(columns=["URL", "Brand", "Name", "Price", "Rating", "Reviews", "Specifications"])

In [18]:
current_url = driver.current_url

In [ ]:
driver.get(current_url)

In [19]:
main_part = driver.find_element(By.XPATH, "/html/body/div/div/div[3]/div[1]/div[2]")
clothes = main_part.find_elements(By.CLASS_NAME, "IRpwTa")
links = []
for link in clothes:
    links.append(link.get_attribute('href'))

In [21]:
def get_details(url, i):
    driver.get(url)
    metadata = {}
    try:
        metadata["URL"] = url
        metadata["Brand"] = driver.find_element(By.CLASS_NAME, "G6XhRU").text
        metadata["Name"] = driver.find_element(By.CLASS_NAME, "B_NuCI").text
        metadata["Price"] = driver.find_element(By.CLASS_NAME, "_25b18c").text
        metadata["Rating"] = driver.find_element(By.CLASS_NAME, "_3_L3jD").text
        metadata["Reviews"] = driver.find_element(By.CLASS_NAME, "_2_R_DZ").text
        driver.find_element(By.CLASS_NAME, "_2yIA0Y").click()
        buttons = driver.find_elements(By.TAG_NAME, "button")
        for button in buttons:
            if button.text == "Read More":
                button.click()
                break
        product = driver.find_element(By.CLASS_NAME, "X3BRps")
        data = product.find_elements(By.CLASS_NAME, "row")
        dict = {}
        for row in data:
            pair = row.text.split('\n')
            dict[pair[0]] = pair[1]
        metadata["Specifications"] = str(dict)
        images = driver.find_element(By.CLASS_NAME, "_2mLllQ")
        image_links = images.find_elements(By.TAG_NAME, 'img')
        for count, image in enumerate(image_links):
            link = image.get_attribute('src')
            better_link = link.replace('128/128', '832/832')
            with open(f'flipkart_images\\{i}_{metadata["Name"]}_{count}.jpg', 'wb') as f:
                f.write(requests.get(better_link).content)
        return metadata
    except Exception as e:
        print("Skipped")

In [22]:
for i, link in enumerate(links):
    metadata = get_details(link, i)
    df = pd.concat([df, pd.DataFrame(metadata, index=[0])], ignore_index=True)

Skipped https://www.flipkart.com/selvia-edge-casual-printed-women-multicolor-top/p/itmbeb88df78c89d?pid=TOPGRVBQFWXJ8UAV&lid=LSTTOPGRVBQFWXJ8UAVDFI4YB&marketplace=FLIPKART&q=pinterest+trendy+crop+tops&store=clo&srno=s_1_2&otracker=search&otracker1=search&fm=organic&iid=en_EVwsjlZDL_VpVV7bv7k0TK9b9e5W6q5H0ksxpBd1QIrUjFDhbDdzUyrhwreguNhmh_gqWoCIZQepXIwNBEMFQQ%3D%3D&ppt=hp&ppn=homepage&ssid=1n1zda3yzk0000001692470837087&qH=6045590b3632ba18 due to Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3_L3jD"}
  (Session info: chrome=115.0.5790.173)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x008FA813+48355]
	(No symbol) [0x0088C4B1]
	(No symbol) [0x00795358]
	(No symbol) [0x007C09A5]
	(No symbol) [0x007C0B3B]
	(No symbol) [0x007EE232]
	(No symbol) [0x007DA784]
	(No symbol) [0x007EC922]
	(No symbol) [0x007DA536]
	(No symbol) [0x007B82DC]
	(No symbol) [0x007B93DD]
	GetHandleVerifier [0x00B5AABD+2539405]
	GetHandleVerifier [0x00B9A78F+2800735]
	GetHandleV

In [23]:
df.to_csv('Girls Tops.csv', index=False)